In [1]:
import pandas as pd
import fornax
from sqlalchemy import create_engine
from sqlalchemy.orm.session import Session

# Tutorial 1 - Populating the Database

In order to use fornax we need to insert the data into a relational database with the correct schema.

It should be noted that all of these steps can be reproduced without fornax by interacting with the database directly.
The equivalent SQL commands are shown in the logging

## Initialise the database
Fortunatly the fornax library can be used to create initialise database using a sqlalchemy connection.

In [2]:
engine = create_engine(
    'sqlite://', # create an in memory sqlite database
    echo=True    # turn logging on 
)
connection = engine.connect()

# create the database schema
fornax.model.Base.metadata.create_all(connection)

2018-09-14 11:21:13,079 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-09-14 11:21:13,079 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-09-14 11:21:13,080 INFO sqlalchemy.engine.base.Engine ()
2018-09-14 11:21:13,080 INFO sqlalchemy.engine.base.Engine ()
2018-09-14 11:21:13,081 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-09-14 11:21:13,081 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-09-14 11:21:13,082 INFO sqlalchemy.engine.base.Engine ()
2018-09-14 11:21:13,082 INFO sqlalchemy.engine.base.Engine ()
2018-09-14 11:21:13,083 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("match")
2018-09-14 11:21:13,083 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("match")
2018-09-14 11:21:13,084 INFO sqlalchemy.engine.base.Engine ()
2018-09-14 11:21:13,084 INFO sqlalchemy.en

## Inserting the nodes

To add the nodes create a `fornax.model.TargetNode` object for each node.

In [8]:
nodes_df = pd.read_csv('./nodes.csv')
# create a list of TargetNode objects
nodes = [fornax.model.TargetNode(id=uid, type=type_) for uid, type_ in zip(nodes_df['uid'], nodes_df['type'])]

In [4]:
session = Session(connection)
session.add_all(nodes)
session.commit()

2018-09-14 11:21:13,206 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-09-14 11:21:13,206 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-09-14 11:21:13,223 INFO sqlalchemy.engine.base.Engine INSERT INTO target_node (id, type) VALUES (?, ?)
2018-09-14 11:21:13,223 INFO sqlalchemy.engine.base.Engine INSERT INTO target_node (id, type) VALUES (?, ?)
2018-09-14 11:21:13,224 INFO sqlalchemy.engine.base.Engine ((2861295873, 0), (2169370700, 0), (421230664, 0), (1747963121, 0), (1390977948, 0), (3458194944, 0), (1036549429, 0), (3036238920, 0)  ... displaying 10 of 1129 total bound parameter sets ...  (1567195514, 2), (580296626, 2))
2018-09-14 11:21:13,224 INFO sqlalchemy.engine.base.Engine ((2861295873, 0), (2169370700, 0), (421230664, 0), (1747963121, 0), (1390977948, 0), (3458194944, 0), (1036549429, 0), (3036238920, 0)  ... displaying 10 of 1129 total bound parameter sets ...  (1567195514, 2), (580296626, 2))
2018-09-14 11:21:13,244 INFO sqlalchemy.engine.base.Engine C

## Inserting the edges

To add the edges create a fornax.model.TargetEdge object for each node.

In [5]:
edges_df = pd.read_csv('./edges.csv')
# create a list of TargetEdge objects
edges = [fornax.model.TargetEdge(start=start, end=end) for start, end in zip(edges_df['start'], edges_df['end'])]

In [6]:
session = Session(connection)
session.add_all(edges)
session.commit()
session.close()

2018-09-14 11:21:13,400 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-09-14 11:21:13,400 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-09-14 11:21:13,436 INFO sqlalchemy.engine.base.Engine INSERT INTO target_edge (start, "end") VALUES (?, ?)
2018-09-14 11:21:13,436 INFO sqlalchemy.engine.base.Engine INSERT INTO target_edge (start, "end") VALUES (?, ?)
2018-09-14 11:21:13,437 INFO sqlalchemy.engine.base.Engine ((2861295873, 694696789), (2169370700, 3821667966), (421230664, 989667430), (421230664, 1588981747), (1747963121, 511082496), (1390977948, 989667430), (3458194944, 2118744719), (1036549429, 1681727299)  ... displaying 10 of 2752 total bound parameter sets ...  (1567195514, 1581878622), (580296626, 3212541426))
2018-09-14 11:21:13,437 INFO sqlalchemy.engine.base.Engine ((2861295873, 694696789), (2169370700, 3821667966), (421230664, 989667430), (421230664, 1588981747), (1747963121, 511082496), (1390977948, 989667430), (3458194944, 2118744719), (1036549429, 1681

Next we add the target edges, the connections between target nodes

In [7]:
connection.close()